In [1]:
import pandas as pd
import torch
import numpy as np

In [15]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
from sklearn.preprocessing import LabelEncoder
import numpy as np
import joblib

In [17]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.3/28.3 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.4/647.4 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 7.9 MB/s eta 0:00:00


In [18]:
import mlflow
import mlflow.transformers

In [12]:
from sklearn.metrics import accuracy_score, f1_score, recall_score

In [2]:
train=pd.read_csv("/content/final_train.csv")
val=pd.read_csv("/content/val.csv")

In [3]:
X_train=train['mail']
X_val=val['mail']
y_train=train['type']
y_val=val['type'].map({'Phishing Email': 1, 'Safe Email': 0})

In [5]:

# Convert Series to list and ensure all inputs are strings for both train and validation
X_train_list = X_train.fillna('').astype(str).tolist()
X_val_list = X_val.fillna('').astype(str).tolist()

In [6]:
# Initialize the tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Tokenize both training and validation data
train_encodings = tokenizer(
    X_train_list,
    truncation=True,
    padding=True,
    max_length=512,
    return_tensors='pt'
)

val_encodings = tokenizer(
    X_val_list,
    truncation=True,
    padding=True,
    max_length=512,
    return_tensors='pt'
)


In [8]:
# Encode the labels for both train and validation
label_encoder = LabelEncoder()
# Fit on training data only to avoid data leakage
y_train_encoded = label_encoder.fit_transform(y_train)
# Transform validation data using the fitted encoder
y_val_encoded = label_encoder.transform(y_val)

In [9]:
class EmailDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {
            key: val[idx].clone().detach()
            for key, val in self.encodings.items()
        }
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [10]:

# Create both training and validation datasets
train_dataset = EmailDataset(train_encodings, y_train_encoded)
val_dataset = EmailDataset(val_encodings, y_val_encoded)


In [11]:
# Set training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,  # Added eval batch size
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",  # Use eval loss to determine best model
    save_total_limit=2,  # Keep only the last 2 checkpoints
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
# Define metrics computation function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, pos_label=1)  # F1-score for class 1
    recall = recall_score(labels, preds, pos_label=1)  # Recall for class 1

    return {
        'accuracy': acc,
        'f1_score_class_1': f1,
        'recall_class_1': recall
    }

In [14]:
# Initialize the Trainer with evaluation dataset
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,  # Added metrics computation
)


In [19]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sudhanvaiitr (sudhanvaiitr-indian-institute-of-technology-roorkee) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score Class 1,Recall Class 1
1,0.094500,0.092325,0.972118,0.965654,0.997271
2,0.006800,0.078250,0.979088,0.973631,0.982265


TrainOutput(global_step=3654, training_loss=0.08202436576732733, metrics={'train_runtime': 1539.8994, 'train_samples_per_second': 18.975, 'train_steps_per_second': 2.373, 'total_flos': 3870697388728320.0, 'train_loss': 0.08202436576732733, 'epoch': 2.0})

In [20]:

# Train the model

# Get the evaluation results
eval_results = trainer.evaluate()
print("\nValidation Results:", eval_results)

# Save the label encoder for future use

joblib.dump(label_encoder, './results/label_encoder.joblib')

# Save the final model
model.save_pretrained('./results/final_model')
tokenizer.save_pretrained('./results/final_model')


Validation Results: {'eval_loss': 0.07824951410293579, 'eval_accuracy': 0.9790884718498659, 'eval_f1_score_class_1': 0.973630831643002, 'eval_recall_class_1': 0.9822646657571623, 'eval_runtime': 26.2679, 'eval_samples_per_second': 70.999, 'eval_steps_per_second': 8.908, 'epoch': 2.0}


('./results/final_model/tokenizer_config.json',
 './results/final_model/special_tokens_map.json',
 './results/final_model/vocab.txt',
 './results/final_model/added_tokens.json')

In [25]:
print('accuracy :',eval_results['eval_accuracy'])
print('f1_score :',eval_results['eval_f1_score_class_1'])
print('recall_class_1 :',eval_results['eval_recall_class_1'])

accuracy : 0.9790884718498659
f1_score : 0.973630831643002
recall_class_1 : 0.9822646657571623
